In [ ]:
#
# https://snap.stanford.edu/data/egonets-Facebook.html
#
# J. McAuley and J. Leskovec. Learning to Discover Social Circles in Ego Networks. NIPS, 2012.
#
import polars as pl
import pandas as pd
import numpy as np
import networkx as nx
import os
from rtsvg import *
rt = RACETrack()

In [ ]:
_base_network_ = '0'
_base_dir_     = '../../../data/stanford/facebook/'
_layout_file_  = _base_dir_ + _base_network_ + '.layout.parquet'
_edges_ = open(_base_dir_ + _base_network_ + '.edges', 'rt').read()
_lu_ = {'fm':[], 'to':[]}
for _edge_ in _edges_.split('\n'):
    if _edge_ == '': continue
    _lu_['fm'].append(_edge_.split(' ')[0])
    _lu_['to'].append(_edge_.split(' ')[1])
df = pl.DataFrame(_lu_)
len(df), len(set(df['fm'])), len(set(df['to'])), len(set(df['fm']) | set(df['to']))

In [3]:
relates = [('fm', 'to')]
g   = rt.createNetworkXGraph(df, relates)
pos = nx.spring_layout(g)

In [ ]:
ln_params = {'relationships':relates, 'pos':pos, 'draw_labels':False, 'node_size':'small'}
_rtg_ = rt.interactiveGraphPanel(df, ln_params, w=800, h=600)
if os.path.exists(_layout_file_): _rtg_.loadLayout(_layout_file_)
_rtg_

In [12]:
#_rtg_.saveLayout(_layout_file_)
# circles from the 0.circles file...
#_rtg_.selectEntities(['71', '215', '54', '61', '298', '229', '81', '253', '193', '97', '264', '29', '132', '110', '163', '259', '183', '334', '245', '222'])
#_rtg_.selectEntities(['155', '99', '327', '140', '116', '147', '144', '150', '270'])

In [6]:
#
# So... the circles files have nodes that exist in more than one circle
#
_circles_ = open(_base_dir_ + _base_network_ + '.circles', 'rt').read()
_already_seen_ = set()
_circle_lu_ = {'name':[], 'node':[]}
for _circle_ in _circles_.split('\n'):
    _circle_name_ = _circle_.split('\t')[0]
    if _circle_ == '': continue
    _set_ = set(_circle_.split('\t')[1:])
    for x in _set_:
        _circle_lu_['node'].append(x), _circle_lu_['name'].append(_circle_name_)
    _already_seen_ = _already_seen_ | _set_
len(_already_seen_)
df_circles = pd.DataFrame(_circle_lu_)
# rt.histogram(df_circles, bin_by='node', count_by='name', h=768)

In [ ]:
_counts_ = {}
def _updateCount_(a,b):
    if a not in _counts_: _counts_[a] = {}
    if b not in _counts_: _counts_[b] = {}
    if b not in _counts_[a]: _counts_[a][b] = 0
    _counts_[a][b] += 1
    if a not in _counts_[b]: _counts_[b][a] = 0
    _counts_[b][a] += 1
for x in nx.simple_cycles(g,3):
    n0, n1, n2 = x
    _updateCount_(n0,n1), _updateCount_(n1,n2), _updateCount_(n0,n2)
_max_ = 1
_lu_  = {'fm':[],'to':[], 'ct':[]}
edges_found = set()
for x in _counts_:
    for y in _counts_[x]:
        # only need to include one direction since this is an undirected graph
        if x < y:
            _lu_['fm'].append(x), _lu_['to'].append(y), _lu_['ct'].append(_counts_[x][y])
            _max_ = max(_max_, _counts_[x][y])
            edges_found.add((x,y)), edges_found.add((y,x))
# Add in any edges that were not found
for _pair_, _df_ in df.group_by(('fm','to')):
    if (_pair_[0],_pair_[1]) not in edges_found:
        _lu_['fm'].append(_pair_[0]), _lu_['to'].append(_pair_[1]), _lu_['ct'].append(_max_)
        edges_found.add((_pair_[0],_pair_[1])), edges_found.add((_pair_[1],_pair_[0]))
rt.xy(pl.DataFrame(_lu_), x_field='ct', y_field='ct', dot_size='vary')

In [ ]:
#
# NOT LIKE THIS
#
df_test_d  = pd.DataFrame({'fm':['a','b','c'], 'to':['b','c','a'], 'ct':[1, 20, 25]})
g_test_d   = rt.createNetworkXGraph(df_test_d, [('fm','to')], count_by='ct')
test_d_pos = nx.spring_layout(g_test_d)

#
# BUT LIKE THIS INSTEAD ... weights need to be inverted ... i.e., 1.0 will be close... 1.0/20.0 will be far
#
df_test_d2  = pd.DataFrame({'fm':['a','b','c'], 'to':['b','c','a'], 'ct':[1.0/1.0, 1.0/20.0, 1.0/25.0]})
g_test_d2   = rt.createNetworkXGraph(df_test_d2, [('fm','to')], count_by='ct')
test_d2_pos = nx.spring_layout(g_test_d2)

rt.tile([rt.link(df_test_d,  [('fm','to')], test_d_pos,  draw_labels=True, bounds_percent=0.2),
         rt.link(df_test_d2, [('fm','to')], test_d2_pos, draw_labels=True, bounds_percent=0.2)])

In [ ]:
df_w  = pl.DataFrame(_lu_)
df_w  = df_w.with_columns((1.0/(1.0+pl.col('ct'))).alias('ct_inv'))
g_w   = rt.createNetworkXGraph(df_w, relates, count_by='ct_inv')
pos_w = nx.spring_layout(g_w, iterations=500)
ln_w_params = {'relationships':relates, 'pos':pos_w, 'draw_labels':False, 'bounds_percent':0.02}
_rtg_w_ = rt.interactiveGraphPanel(df_w, ln_w_params, w=512, h=512)
_rtg_w_